## Get Data

In [1]:
!pip install ultralytics==8.0.196
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 99.0 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 99.7 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 MB 67.3 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 77.0 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 75.7 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import sys
import subprocess
from roboflow import Roboflow
from concurrent.futures import ThreadPoolExecutor


  0%|          | 0.00/131M [00:00<?, ?B/s]
  3%|▎         | 3.62M/131M [00:00<00:04, 32.5MB/s]
 13%|█▎        | 16.8M/131M [00:00<00:01, 89.9MB/s]
 21%|██▏       | 28.0M/131M [00:00<00:01, 102MB/s] 
 30%|███       | 39.2M/131M [00:00<00:00, 108MB/s]
 39%|███▊      | 50.5M/131M [00:00<00:00, 111MB/s]
 47%|████▋     | 61.9M/131M [00:00<00:00, 114MB/s]
 56%|█████▌    | 73.1M/131M [00:00<00:00, 115MB/s]
 65%|██████▍   | 84.4M/131M [00:00<00:00, 116MB/s]
 73%|███████▎  | 95.5M/131M [00:00<00:00, 109MB/s]
 81%|████████  | 106M/131M [00:01<00:00, 92.8MB/s]
 88%|████████▊ | 115M/131M [00:01<00:00, 83.6MB/s]
 95%|█████████▍| 124M/131M [00:01<00:00, 79.2MB/s]
100%|██████████| 131M/131M [00:01<00:00, 92.4MB/s]
/home/aquinate/anaconda3/envs/yolo-soccer/lib/python3.12/site-packages/ultralytics/nn/tasks.py:567: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle

In [2]:
rf = Roboflow(api_key="Z4C8uDsWoaB3f6gOLtsk")
project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to football-players-detection-1 in yolov8:: 100%|


In [3]:
dataset.location

'/home/aquinate/Documents/DataScience/GremioPrudente/training/datasets/football-players-detection-1'

## Train model

In [4]:
def run_yolo_command(dataset_location):
    command = [
        "yolo", 
        "task=detect", 
        "mode=train", 
        "model=yolov8x.pt", 
        f"data={dataset_location}/data.yaml", 
        "epochs=10", 
        "imgsz=640"
    ]
    
    print("Iniciando o comando YOLO...")
    
    try:
        # Executa o comando
        result = subprocess.run(command, capture_output=True, text=True)
        
        # Verifica se o comando foi executado com sucesso
        if result.returncode == 0:
            print("Comando YOLO executado com sucesso.")
        else:
            print("O comando YOLO falhou com o código de retorno:", result.returncode)
        
        # Exibe a saída padrão
        if result.stdout:
            print("Saída do comando:")
            print(result.stdout)
        
        # Exibe os erros, se houver
        if result.stderr:
            print("Erros do comando:", file=sys.stderr)
            print(result.stderr, file=sys.stderr)
    
    except Exception as e:
        print("Ocorreu uma exceção durante a execução do comando:", str(e), file=sys.stderr)

In [5]:
# Define your dataset location
dataset_location = dataset.location

In [6]:
# Crie um ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=10) as executor:
    # Execute o comando YOLO assincronamente
    future = executor.submit(run_yolo_command, dataset_location)
    # Aguarde a conclusão do comando
    future.result()

KeyboardInterrupt: 

In [ ]:
!yolo task=detect mode=train model=yolov8x.pt data={dataset.location}/data.yaml epochs=10 imgsz=640

100%|████████████████████████████████████████| 131M/131M [00:01<00:00, 92.2MB/s]
/home/aquinate/anaconda3/envs/youtube-downloader/lib/python3.12/site-packages/ultralytics/nn/tasks.py:567: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on G

In [ ]:
import subprocess
import sys
from concurrent.futures import ThreadPoolExecutor
import os

def run_yolo_command(dataset_location):
    command = [
        "yolo", 
        "task=detect", 
        "mode=train", 
        "model=yolov8x.pt", 
        f"data={dataset_location}/data.yaml", 
        "epochs=10", 
        "imgsz=640"
    ]
    
    print("Iniciando o comando YOLO...")
    
    try:
        # Executa o comando
        result = subprocess.run(command, capture_output=True, text=True)
        
        # Verifica se o comando foi executado com sucesso
        if result.returncode == 0:
            print("Comando YOLO executado com sucesso.")
        else:
            print("O comando YOLO falhou com o código de retorno:", result.returncode)
        
        # Exibe a saída padrão
        if result.stdout:
            print("Saída do comando:")
            print(result.stdout)
        
        # Exibe os erros, se houver
        if result.stderr:
            print("Erros do comando:", file=sys.stderr)
            print(result.stderr, file=sys.stderr)
    
    except Exception as e:
        print("Ocorreu uma exceção durante a execução do comando:", str(e), file=sys.stderr)

# Defina o local do dataset
dataset_location = "caminho/do/seu/dataset"  # Substitua com o caminho real

# Crie um ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=10) as executor:
    # Execute o comando YOLO assincronamente
    future = executor.submit(run_yolo_command, dataset_location)
    # Aguarde a conclusão do comando
    future.result()
